# Lab06 – Decision Tree from Scratch
### Using dataset: `example_unprocessed.xlsx`
---
This notebook implements all steps A1–A7:
1. Entropy function
2. Gini Index
3. Information Gain & root node selection
4. Binning (equal-width & frequency)
5. Custom Decision Tree construction
6. Tree visualization
7. Decision boundary visualization


In [ ]:

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import Counter
from typing import Any, List, Union
from sklearn.tree import DecisionTreeClassifier

# Load dataset
df = pd.read_excel("example_unprocessed.xlsx")
df.head()


## A1. Entropy Function

In [ ]:

def entropy(labels: Union[pd.Series, List[Any], np.ndarray]) -> float:
    counts = Counter([x for x in labels if pd.notna(x)])
    total = sum(counts.values())
    if total == 0: return 0.0
    return -sum((c/total) * math.log2(c/total) for c in counts.values())

# Example usage on target column
TARGET_COLUMN = df.columns[-1]  # auto-select last column (adjust if needed)
print("Entropy of target:", entropy(df[TARGET_COLUMN]))


## A2. Gini Index

In [ ]:

def gini_index(labels: Union[pd.Series, List[Any], np.ndarray]) -> float:
    counts = Counter([x for x in labels if pd.notna(x)])
    total = sum(counts.values())
    if total == 0: return 0.0
    return 1.0 - sum((c/total)**2 for c in counts.values())

print("Gini Index of target:", gini_index(df[TARGET_COLUMN]))


## A3 & A4. Binning and Information Gain

In [ ]:

def equal_width_binning(series: pd.Series, bins: int = 4):
    binned, edges = pd.cut(series, bins=bins, retbins=True, labels=False, include_lowest=True, duplicates="drop")
    labels = [f"bin_{int(i)}" if pd.notna(i) else np.nan for i in binned]
    return pd.Series(labels, index=series.index, dtype="category"), list(edges)

def frequency_binning(series: pd.Series, bins: int = 4):
    binned, edges = pd.qcut(series, q=bins, retbins=True, labels=False, duplicates="drop")
    labels = [f"bin_{int(i)}" if pd.notna(i) else np.nan for i in binned]
    return pd.Series(labels, index=series.index, dtype="category"), list(edges)

def bin_series(series: pd.Series, bins: int = 4, method: str = "equal_width"):
    if method == "equal_width": return equal_width_binning(series, bins)
    if method == "frequency": return frequency_binning(series, bins)
    raise ValueError("Unknown method")

def information_gain(parent_labels: pd.Series, split_series: pd.Series) -> float:
    parent_entropy = entropy(parent_labels)
    total = len(split_series.dropna())
    if total == 0: return 0.0
    ig = parent_entropy
    for v, idx in split_series.dropna().groupby(split_series).groups.items():
        subset = parent_labels.loc[idx]
        ig -= (len(idx) / total) * entropy(subset)
    return ig

# Example IG for first feature
feat = df.columns[0]
X_binned, _ = bin_series(df[feat], bins=4)
print("Information Gain:", information_gain(df[TARGET_COLUMN], X_binned))


## A5. Custom Decision Tree

In [ ]:

class DTNode:
    def __init__(self, is_leaf=False, prediction=None, feature=None, children=None):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.feature = feature
        self.children = children or {}

def majority_class(labels: pd.Series):
    cnt = Counter(labels.dropna())
    return cnt.most_common(1)[0][0] if cnt else None

def build_decision_tree(data: pd.DataFrame, target: str, features: List[str],
                        bins=4, binning_method="equal_width",
                        max_depth=None, depth=0) -> DTNode:
    y = data[target]
    if len(y.dropna().unique()) <= 1:
        return DTNode(is_leaf=True, prediction=majority_class(y))
    if not features or (max_depth is not None and depth >= max_depth):
        return DTNode(is_leaf=True, prediction=majority_class(y))

    best_feature, best_ig, best_split = None, -1, None
    for feat in features:
        X = data[feat]
        if pd.api.types.is_numeric_dtype(X):
            split_series, _ = bin_series(X, bins=bins, method=binning_method)
        else:
            split_series = X.astype("category")
        ig = information_gain(y, split_series)
        if ig > best_ig:
            best_ig, best_feature, best_split = ig, feat, split_series

    if not best_feature:
        return DTNode(is_leaf=True, prediction=majority_class(y))

    node = DTNode(is_leaf=False, feature=best_feature, children={})
    remaining_features = [f for f in features if f != best_feature]

    for val in best_split.dropna().unique():
        idx = best_split[best_split == val].index
        subset = data.loc[idx]
        node.children[val] = build_decision_tree(
            subset, target, remaining_features, bins, binning_method, max_depth, depth+1
        )
    return node

def print_tree(node: DTNode, indent=""):
    if node.is_leaf:
        print(indent + "Leaf:", node.prediction)
    else:
        print(indent + "Feature:", node.feature)
        for val, child in node.children.items():
            print(indent + f" {val} ->")
            print_tree(child, indent + "    ")

features = [c for c in df.columns if c != TARGET_COLUMN]
custom_tree = build_decision_tree(df.dropna(subset=[TARGET_COLUMN]), TARGET_COLUMN, features)
print_tree(custom_tree)


## A6. Decision Tree Visualization

In [ ]:

# Simple visualization with text output already shown above.
# For a more graphical view, you could export to graphviz or matplotlib.


## A7. Decision Boundary Visualization

In [ ]:

def decision_boundary_plot(data, target, features):
    X = data[features].copy()
    y = data[target].copy()
    for col in X.columns:
        if not pd.api.types.is_numeric_dtype(X[col]):
            X[col] = X[col].astype("category").cat.codes
    if not pd.api.types.is_numeric_dtype(y):
        y = y.astype("category").cat.codes
    clf = DecisionTreeClassifier(criterion="entropy").fit(X.values, y.values)

    x_min, x_max = X.values[:,0].min()-1, X.values[:,0].max()+1
    y_min, y_max = X.values[:,1].min()-1, X.values[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                         np.linspace(y_min, y_max, 300))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X.values[:,0], X.values[:,1], c=y, edgecolors="k")
    plt.xlabel(features[0]); plt.ylabel(features[1])
    plt.title("Decision Boundary")
    plt.show()

if len(features) >= 2:
    decision_boundary_plot(df.dropna(subset=[TARGET_COLUMN]), TARGET_COLUMN, features[:2])
